In [ ]:
[오후 5:10] 정종현
import requests
import gradio as gr
from PIL import Image, ImageDraw, ImageFont
 
def request_vision(image_path):
    features = "SmartCrops"
    endpoint = f"https://fimtrus-computervision2.cognitiveservices.azure.com/computervision/imageanalysis:analyze?api-version=2024-02-01&features={features}&smartcrops-aspect-ratios=1,1.1,1.5"
    api_key = "ee6d3d3de333480291b572fc7147b015"
 
 
    headers = {
        "Content-Type": "application/octet-stream",
        "Ocp-Apim-Subscription-Key": api_key
    }
 
    with open(image_path, 'rb') as image:
        image_data = image.read()
 
    response = requests.post(endpoint,
                            headers=headers,
                            data=image_data)
 
    response_json = response.json()
    print(response_json)
 
    return response_json
 
def change_image(image_path):
 
    if image_path is None:
        return None
    
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)
    try:
        font = ImageFont.truetype("NanumGothic.ttf", 30)
    except IOError:
        # Arial 폰트가 없으면 기본 폰트를 사용합니다.
        font = ImageFont.load_default()
    
    # request_vision을 통해 데이터를 받아 온다.
    # 받은 데이터를 파싱해서 image에 그린다.
 
    response_json = request_vision(image_path)
    # 받은 데이터를 파싱해서 image에 그린다.  
    if "objectsResult" in response_json and "values" in response_json["objectsResult"]:  
        for obj in response_json["objectsResult"]["values"]:  
            box = obj["boundingBox"]  
            x, y, w, h = box["x"], box["y"], box["w"], box["h"]  
            print(x, y, w, h)
            draw.rectangle([(x, y), (x + w, y + h)], outline="red", width=2)  
            if "tags" in obj:  
                for tag in obj["tags"]:  
                    draw.text((x, y - 10), tag["name"], fill="red", font=font)  
 
    if "tagsResult" in response_json and "values" in response_json["tagsResult"]:  
        for obj in response_json["tagsResult"]["values"]:  
            print(obj['name'], int(obj['confidence']*100))
 
    if "smartCropsResult" in response_json and "values" in response_json["smartCropsResult"]:  
        for obj in response_json["smartCropsResult"]["values"]:  
            box = obj["boundingBox"]  
            x, y, w, h = box["x"], box["y"], box["w"], box["h"]  
            print(x, y, w, h)
            draw.rectangle([(x, y), (x + w, y + h)], outline="red", width=2)  
            draw.text((x, y - 10), str(obj['aspectRatio']), fill="red", font=font)  
 
 
    return image
 
with gr.Blocks() as demo:
 
    input_image = gr.Image(label="이미지 선택", type="filepath", width=800)
    output_image = gr.Image(label="출력 이미지", type="pil", width=800, interactive=False)
 
    input_image.change(fn=change_image, inputs=[input_image], outputs=[output_image])
 
demo.launch()
 